# Data Munging

In [1]:
import pandas as pd
import numpy as np

In [2]:
official_boxscore = pd.read_csv('./data/games.csv')
official_boxscore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23195 entries, 0 to 23194
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     23195 non-null  object 
 1   GAME_ID           23195 non-null  int64  
 2   GAME_STATUS_TEXT  23195 non-null  object 
 3   HOME_TEAM_ID      23195 non-null  int64  
 4   VISITOR_TEAM_ID   23195 non-null  int64  
 5   SEASON            23195 non-null  int64  
 6   TEAM_ID_home      23195 non-null  int64  
 7   PTS_home          23096 non-null  float64
 8   FG_PCT_home       23096 non-null  float64
 9   FT_PCT_home       23096 non-null  float64
 10  FG3_PCT_home      23096 non-null  float64
 11  AST_home          23096 non-null  float64
 12  REB_home          23096 non-null  float64
 13  TEAM_ID_away      23195 non-null  int64  
 14  PTS_away          23096 non-null  float64
 15  FG_PCT_away       23096 non-null  float64
 16  FT_PCT_away       23096 non-null  float6

There is some null values in the data_frames, for the key stats like PTS, AST and REB. Besides that, this dataset does not record stats like STL, BLK, TO that will be needed.

After some exploration, I realised there is no game details for the game of the null values in 'game_details.csv' also. Therefore, I decided to drop these rows.

In [3]:
official_boxscore = official_boxscore.dropna(how='any')
official_boxscore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23096 entries, 0 to 23194
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     23096 non-null  object 
 1   GAME_ID           23096 non-null  int64  
 2   GAME_STATUS_TEXT  23096 non-null  object 
 3   HOME_TEAM_ID      23096 non-null  int64  
 4   VISITOR_TEAM_ID   23096 non-null  int64  
 5   SEASON            23096 non-null  int64  
 6   TEAM_ID_home      23096 non-null  int64  
 7   PTS_home          23096 non-null  float64
 8   FG_PCT_home       23096 non-null  float64
 9   FT_PCT_home       23096 non-null  float64
 10  FG3_PCT_home      23096 non-null  float64
 11  AST_home          23096 non-null  float64
 12  REB_home          23096 non-null  float64
 13  TEAM_ID_away      23096 non-null  int64  
 14  PTS_away          23096 non-null  float64
 15  FG_PCT_away       23096 non-null  float64
 16  FT_PCT_away       23096 non-null  float6

Next, load another dataset 'game_details.csv' which contains stats of every player in every game.

In [4]:
game_details = pd.read_csv('./data/games_details.csv')
game_details.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,21900895,1610612749,MIL,Milwaukee,202083,Wesley Matthews,F,NaN,27:08,3.0,...,4.0,4.0,8.0,2.0,2.0,0.0,0.0,0.0,8.0,11.0
1,21900895,1610612749,MIL,Milwaukee,203507,Giannis Antetokounmpo,F,NaN,34:55,17.0,...,2.0,18.0,20.0,6.0,1.0,0.0,3.0,2.0,41.0,22.0
2,21900895,1610612749,MIL,Milwaukee,201572,Brook Lopez,C,NaN,26:25,4.0,...,2.0,5.0,7.0,0.0,0.0,3.0,0.0,2.0,16.0,16.0
3,21900895,1610612749,MIL,Milwaukee,1628978,Donte DiVincenzo,G,NaN,27:35,1.0,...,1.0,6.0,7.0,5.0,0.0,1.0,2.0,0.0,2.0,14.0
4,21900895,1610612749,MIL,Milwaukee,202339,Eric Bledsoe,G,NaN,22:17,2.0,...,1.0,0.0,1.0,2.0,1.0,0.0,3.0,2.0,4.0,6.0


Compute the sum of the stats for each team and add them to a new column for official_boxscore.

In [5]:
def stats_sum(boxscore, details, stats, team):
    game_id = boxscore['GAME_ID']
    if(team=='home'):
        team_id = boxscore['HOME_TEAM_ID']
    else:
        team_id = boxscore['VISITOR_TEAM_ID']
    details_filtered = details[(details['GAME_ID'] == game_id) & (details['TEAM_ID'] == team_id)]
    return np.sum(details_filtered[stats])
    
home_array = ['STL_home','BLK_home','FTM_home','TO_home']
away_array = [i.replace('home','away') for i in home_array]

official_boxscore[home_array] = official_boxscore.apply(stats_sum, args = (game_details, ['STL', 'BLK','FTM', 'TO'], 'home'),axis=1)
official_boxscore[away_array] = official_boxscore.apply(stats_sum, args = (game_details, ['STL', 'BLK','FTM','TO'], 'away'),axis=1)
display(official_boxscore.head())

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,REB_away,HOME_TEAM_WINS,STL_home,BLK_home,FTM_home,TO_home,STL_away,BLK_away,FTM_away,TO_away
0,2020-03-01,21900895,Final,1610612766,1610612749,2019,1610612766,85.0,0.354,0.900,...,61.0,0,7.0,8.0,9.0,10.0,7.0,6.0,16.0,16.0
1,2020-03-01,21900896,Final,1610612750,1610612742,2019,1610612750,91.0,0.364,0.400,...,56.0,0,9.0,3.0,6.0,13.0,10.0,7.0,12.0,14.0
2,2020-03-01,21900897,Final,1610612746,1610612755,2019,1610612746,136.0,0.592,0.805,...,37.0,1,6.0,6.0,33.0,10.0,7.0,4.0,13.0,9.0
3,2020-03-01,21900898,Final,1610612743,1610612761,2019,1610612743,133.0,0.566,0.700,...,36.0,1,6.0,1.0,21.0,16.0,9.0,4.0,26.0,11.0
4,2020-03-01,21900899,Final,1610612758,1610612765,2019,1610612758,106.0,0.407,0.885,...,42.0,1,8.0,7.0,23.0,10.0,6.0,4.0,16.0,14.0


Rearrange the columns and output to csv.

In [6]:
columns = list(official_boxscore.columns)
print(columns)
reindex_columns = ['SEASON'] + columns[7:13] + columns[21:25] + columns[14:20] + columns[25:] + [columns[20]]
print(reindex_columns)

['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS', 'STL_home', 'BLK_home', 'FTM_home', 'TO_home', 'STL_away', 'BLK_away', 'FTM_away', 'TO_away']
['SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'STL_home', 'BLK_home', 'FTM_home', 'TO_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 'STL_away', 'BLK_away', 'FTM_away', 'TO_away', 'HOME_TEAM_WINS']


In [7]:
official_boxscore = official_boxscore[reindex_columns]

In [8]:
# for col in official_boxscore.columns:
#     if official_boxscore[col].dtype == np.float64:
#         official_boxscore[col] = pd.to_numeric(official_boxscore[col], downcast='float')
#     elif official_boxscore[col].dtype == np.int64:
#         official_boxscore[col] = pd.to_numeric(official_boxscore[col], downcast='unsigned')


In [9]:
official_boxscore.to_csv('./output_data/detailed_boxscore.csv')